# TFX Components Walk-through

## Learning Objectives

1.  Develop a high level understanding of TFX pipeline components.
2.  Learn how to use a TFX Interactive Context for prototype development of TFX pipelines.
3.  Work with the Tensorflow Data Validation (TFDV) library to check and analyze input data.
4.  Utilize the Tensorflow Transform (TFT) library for scalable data preprocessing and feature transformations.
5.  Employ the Tensorflow Model Analysis (TFMA) library for model evaluation.

In this lab, you will work with the [Covertype Data Set](https://github.com/jarokaz/mlops-labs/blob/master/datasets/covertype/README.md) and use TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features.

You will utilize  **TFX Interactive Context** to work with the TFX components interactivelly in a Jupyter notebook environment. Working in an interactive notebook is useful when doing initial data exploration, experimenting with models, and designing ML pipelines. You should be aware that there are differences in the way interactive notebooks are orchestrated, and how they access metadata artifacts. In a production deployment of TFX on GCP, you will use an orchestrator such as Kubeflow Pipelines, or Cloud Composer. In an interactive mode, the notebook itself is the orchestrator, running each TFX component as you execute the notebook cells. In a production deployment, ML Metadata will be managed in a scalabe database like MySQL, and artifacts in apersistent store such as Google Cloud Storage. In an interactive mode, both properties and payloads are stored in a local file system of the Jupyter host.

**Setup Note:**
Currently, TFMA visualizations do not render properly in JupyterLab. It is recommended to run this notebook in Jupyter Classic Notebook. To switch to Classic Notebook select *Launch Classic Notebook* from the *Help* menu.

In [1]:
import absl
import os
import tempfile
import time

import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
import tfx

from pprint import pprint
from tensorflow_metadata.proto.v0 import schema_pb2, statistics_pb2, anomalies_pb2
from tensorflow_transform.tf_metadata import schema_utils
from tfx.components import CsvExampleGen
from tfx.components import Evaluator
from tfx.components import ExampleValidator
from tfx.components import InfraValidator
from tfx.components import Pusher
from tfx.components import ResolverNode
from tfx.components import SchemaGen
from tfx.components import StatisticsGen
from tfx.components import Trainer
from tfx.components import Transform
from tfx.components import Tuner
from tfx.dsl.components.base import executor_spec
from tfx.components.common_nodes.importer_node import ImporterNode
from tfx.components.trainer import executor as trainer_executor
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.orchestration import metadata
from tfx.orchestration import pipeline
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import example_gen_pb2
from tfx.proto import infra_validator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.proto.evaluator_pb2 import SingleSlicingSpec

from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import HyperParameters
from tfx.types.standard_artifacts import ModelBlessing
from tfx.types.standard_artifacts import InfraBlessing

2021-11-10 07:21:04.186539: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-10 07:21:04.186583: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


**Note**: this lab was developed and tested with the following TF ecosystem package versions:

`Tensorflow Version: 2.3.1`  
`TFX Version: 0.25.0`  
`TFDV Version: 0.25.0`  
`TFMA Version: 0.25.0`

If you encounter errors with the above imports (e.g. TFX component not found), check your package versions in the cell below.

In [2]:
print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)
print("TFDV Version:", tfdv.__version__)
print("TFMA Version:", tfma.VERSION_STRING)

absl.logging.set_verbosity(absl.logging.INFO)

Tensorflow Version: 2.3.1
TFX Version: 0.25.0
TFDV Version: 0.25.0
TFMA Version: 0.25.0


If the versions above do not match, update your packages in the current Jupyter kernel below. The default `%pip` package installation location is not on your system installation PATH; use the command below to append the local installation path to pick up the latest package versions. Note that you may also need to restart your notebook kernel to pick up the specified package versions and re-run the imports cell above before proceeding with the lab.

In [3]:
os.environ['PATH'] += os.pathsep + '/home/jupyter/.local/bin'

## Configure lab settings

Set constants, location paths and other environment settings. 

In [4]:
ARTIFACT_STORE = os.path.join(os.sep, 'home', 'jupyter', 'artifact-store')
SERVING_MODEL_DIR=os.path.join(os.sep, 'home', 'jupyter', 'serving_model')
DATA_ROOT = 'gs://workshop-datasets/covertype/small'

## Creating Interactive Context

TFX Interactive Context allows you to create and run TFX Components in an interactive mode. It is designed to support experimentation and development in a Jupyter Notebook environment. It is an experimental feature and major changes to interface and functionality are expected. When creating the interactive context you can specifiy the following parameters:
- `pipeline_name` - Optional name of the pipeline for ML Metadata tracking purposes. If not specified, a name will be generated for you.
- `pipeline_root` - Optional path to the root of the pipeline's outputs. If not specified, an ephemeral temporary directory will be created and used.
- `metadata_connection_config` - Optional `metadata_store_pb2.ConnectionConfig` instance used to configure connection to a ML Metadata connection. If not specified, an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.


In [5]:
PIPELINE_NAME = 'tfx-covertype-classifier'
PIPELINE_ROOT = os.path.join(ARTIFACT_STORE, PIPELINE_NAME, time.strftime("%Y%m%d_%H%M%S"))
os.makedirs(PIPELINE_ROOT, exist_ok=True)

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=PIPELINE_ROOT,
    metadata_connection_config=None)

## Ingesting data using ExampleGen

In any ML development process the first step  is to ingest the training and test datasets. The `ExampleGen` component ingests data into a TFX pipeline. It consumes external files/services to generate a set file files in the `TFRecord` format,  which will be used by other TFX components. It can also shuffle the data and split into an arbitrary number of partitions.

<img src=../../images/ExampleGen.png width="300">

### Configure and run CsvExampleGen

In this exercise, you use the `CsvExampleGen` specialization of `ExampleGen` to ingest CSV files from a GCS location and emit them as `tf.Example` records for consumption by downstream TFX pipeline components. Your task is to configure the component to create 80-20 `train` and `eval` splits. *Hint: review the [ExampleGen proto](https://github.com/tensorflow/tfx/blob/master/tfx/proto/example_gen.proto#L78) definition to split your data with hash buckets.*

In [7]:
output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(
        splits=[
            example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
            example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
        ]
    )
)

example_gen = tfx.components.CsvExampleGen(
    input_base=DATA_ROOT,
    output_config=output_config
)

In [8]:
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data gs://workshop-datasets/covertype/small/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:5511886,xor_checksum:1585681978,sum_checksum:1585681978"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            string_value: "0"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        ))

### Examine the ingested data

In [9]:
examples_uri = example_gen.outputs['examples'].get()[0].uri
tfrecord_filenames = [os.path.join(examples_uri, 'train', name)
                      for name in os.listdir(os.path.join(examples_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

Soil_Type: [b'C7757']
Cover_Type: [1]
Horizontal_Distance_To_Hydrology: [648]
Horizontal_Distance_To_Roadways: [757]
Wilderness_Area: [b'Commanche']
Hillshade_3pm: [157]
Horizontal_Distance_To_Fire_Points: [1871]
Elevation: [3142]
Slope: [9]
Vertical_Distance_To_Hydrology: [101]
Hillshade_Noon: [247]
Hillshade_9am: [223]
Aspect: [183]
******
Wilderness_Area: [b'Cache']
Horizontal_Distance_To_Roadways: [124]
Vertical_Distance_To_Hydrology: [9]
Hillshade_9am: [245]
Aspect: [124]
Horizontal_Distance_To_Hydrology: [60]
Horizontal_Distance_To_Fire_Points: [451]
Hillshade_Noon: [227]
Elevation: [1967]
Slope: [16]
Cover_Type: [2]
Soil_Type: [b'C2704']
Hillshade_3pm: [105]
******


2021-11-10 07:25:14.404986: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-10 07:25:14.405159: W tensorflow/stream_executor/cuda/cuda_driver.cc:312] failed call to cuInit: UNKNOWN ERROR (303)
2021-11-10 07:25:14.405281: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (tensorflow-2-3-20211101-161051): /proc/driver/nvidia/version does not exist
2021-11-10 07:25:14.406423: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flag

## Generating statistics using StatisticsGen

The `StatisticsGen`  component generates data statistics that can be used by other TFX components. StatisticsGen uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started). `StatisticsGen` generate statistics for each split in the `ExampleGen` component's output. In our case there two splits: `train` and `eval`.

<img src=../../images/StatisticsGen.png width="200">

### Configure and  run the `StatisticsGen` component

In [10]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])

INFO:absl:Excluding no splits because exclude_splits is not set.


In [11]:
context.run(statistics_gen)

INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/StatisticsGen/statistics/2/train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/StatisticsGen/statistics/2/eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

### Visualize statistics

The generated statistics can be visualized using the `tfdv.visualize_statistics()` function from the [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started) library or using a utility method of the `InteractiveContext` object. In fact, most of the artifacts generated by the TFX components can be visualized using `InteractiveContext`.

In [12]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


## Infering data schema using SchemaGen

Some TFX components use a description input data called a schema. The schema is an instance of `schema.proto`. It can specify data types for feature values, whether a feature has to be present in all examples, allowed value ranges, and other properties. `SchemaGen` automatically generates the schema by inferring types, categories, and ranges from data statistics. The auto-generated schema is best-effort and only tries to infer basic properties of the data. It is expected that developers review and modify it as needed. `SchemaGen` uses [TensorFlow Data Validation](https://www.tensorflow.org/tfx/data_validation/get_started).

The `SchemaGen` component generates the schema using the statistics for the `train` split. The statistics for other splits are ignored.

<img src=../../images/SchemaGen.png width="200">

### Configure and run the `SchemaGen` components

In [13]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)

INFO:absl:Excluding no splits because exclude_splits is not set.


In [14]:
context.run(schema_gen)

INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
2021-11-10 07:27:47.915970: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

### Visualize the inferred schema

In [15]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,-
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Updating the auto-generated schema

In most cases the auto-generated schemas must be fine-tuned manually using insights from data exploration and/or domain knowledge about the data. For example, you know that in the `covertype` dataset there are seven types of forest cover (coded using 1-7 range) and that the value of the `Slope` feature should be in the 0-90 range. You can manually add these constraints to the auto-generated schema by setting the feature domain.

### Load the auto-generated schema proto file

In [16]:
schema_proto_path = '{}/{}'.format(schema_gen.outputs['schema'].get()[0].uri, 'schema.pbtxt')
schema = tfdv.load_schema_text(schema_proto_path)

### Modify the schema

You can use the protocol buffer APIs to modify the schema. 

**Hint**: Review the [TFDV library API documentation](https://www.tensorflow.org/tfx/data_validation/api_docs/python/tfdv/set_domain) on setting a feature's domain. You can use the protocol buffer APIs to modify the schema. Review the [Tensorflow Metadata proto definition](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto#L405) for configuration options.

In [17]:
# TODO: Your code to restrict the categorical feature Cover_Type between the values of 0 and 6.
# TODO: Your code to restrict the numeric feature Slope between 0 and 90.
tfdv.set_domain(schema, 'Cover_Type', schema_pb2.IntDomain(name='Cover_Type', min=0, max=6, is_categorical=True))
tfdv.set_domain(schema, 'Slope',  schema_pb2.IntDomain(name='Slope', min=0, max=90))

tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,"[0,6]"
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


#### Save the updated schema 

In [18]:
schema_dir = os.path.join(ARTIFACT_STORE, 'schema')
tf.io.gfile.makedirs(schema_dir)
schema_file = os.path.join(schema_dir, 'schema.pbtxt')

tfdv.write_schema_text(schema, schema_file)

!cat {schema_file}

feature {
  name: "Aspect"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Cover_Type"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  int_domain {
    name: "Cover_Type"
    min: 0
    max: 6
    is_categorical: true
  }
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Elevation"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_3pm"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_9am"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Hillshade_Noon"
  value_count {
    min: 1
    max: 1
  }
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
}
feature {
  name: "Horizontal_Distance_To_Fire_P

## Importing the updated schema using ImporterNode

The `ImporterNode` component allows you to import an external artifact, including the schema file, so it can be used by other TFX components in your workflow. 


### Configure and run the `ImporterNode` component

In [19]:
schema_importer = ImporterNode(
    instance_name='Schema_Importer',
    source_uri=schema_dir,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False
)

In [20]:
context.run(schema_importer)

INFO:absl:Running driver for ImporterNode.Schema_Importer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: /home/jupyter/artifact-store/schema, properties: {}, custom_properties: {}
INFO:absl:Running executor for ImporterNode.Schema_Importer
INFO:absl:Running publisher for ImporterNode.Schema_Importer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 4
        type_id: 9
        uri: "/home/jupyter/artifact-store/schema"
        , artifact_type: id: 9
        name: "Schema"
        )]
        ))

### Visualize the imported schema

In [21]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'Aspect',INT,required,single,-
'Cover_Type',INT,required,single,"[0,6]"
'Elevation',INT,required,single,-
'Hillshade_3pm',INT,required,single,-
'Hillshade_9am',INT,required,single,-
'Hillshade_Noon',INT,required,single,-
'Horizontal_Distance_To_Fire_Points',INT,required,single,-
'Horizontal_Distance_To_Hydrology',INT,required,single,-
'Horizontal_Distance_To_Roadways',INT,required,single,-


/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:151: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'Soil_Type',"'C2702', 'C2703', 'C2704', 'C2705', 'C2706', 'C2717', 'C3501', 'C3502', 'C4201', 'C4703', 'C4704', 'C4744', 'C4758', 'C5101', 'C5151', 'C6101', 'C6102', 'C6731', 'C7101', 'C7102', 'C7103', 'C7201', 'C7202', 'C7700', 'C7701', 'C7702', 'C7709', 'C7710', 'C7745', 'C7746', 'C7755', 'C7756', 'C7757', 'C7790', 'C8703', 'C8707', 'C8708', 'C8771', 'C8772', 'C8776'"
'Wilderness_Area',"'Cache', 'Commanche', 'Neota', 'Rawah'"


## Validating data with ExampleValidator

The `ExampleValidator` component identifies anomalies in data.  It identifies anomalies by comparing data statistics computed by the `StatisticsGen` component against a schema generated by `SchemaGen` or imported by `ImporterNode`.

`ExampleValidator` can detect different classes of anomalies. For example it can:

- perform validity checks by comparing data statistics against a schema 
- detect training-serving skew by comparing training and serving data.
- detect data drift by looking at a series of data.


The `ExampleValidator` component validates the data in the `eval` split only. Other splits are ignored. 

<img src=../../images/ExampleValidator.png width="350">

### Configure and run the `ExampleValidator` component


In [22]:
# TODO: Complete ExampleValidator
# Hint: review the visual above and review the documentation on ExampleValidator's inputs and outputs: 
# https://www.tensorflow.org/tfx/guide/exampleval
# Make sure you use the output of the schema_importer component created above.

example_validator = ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_gen.outputs['schema']
)

INFO:absl:Excluding no splits because exclude_splits is not set.


In [23]:
context.run(example_validator)

INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/ExampleValidator/anomalies/5/train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/ExampleValidator/anomalies/5/eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 5
        type_id: 12
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/ExampleValidator/anomalies/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 12
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
        ))

### Examine the output of `ExampleValidator`

The output artifact of the ExampleValidator is the `anomalies.pbtxt` file describing an anomalies_pb2.Anomalies protobuf.

In [24]:
train_uri = example_validator.outputs['anomalies'].get()[0].uri
train_anomalies_filename = os.path.join(train_uri, "train/anomalies.pbtxt")
!cat $train_anomalies_filename

baseline {
  feature {
    name: "Aspect"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Cover_Type"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Elevation"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Hillshade_3pm"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Hillshade_9am"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_count: 1
    }
  }
  feature {
    name: "Hillshade_Noon"
    value_count {
      min: 1
      max: 1
    }
    type: INT
    presence {
      min_fraction: 1.0
      min_c

### Visualize validation results

The file `anomalies.pbtxt` can be visualized using `context.show`.

In [25]:
context.show(example_validator.outputs['output'])

In our case no anomalies were detected in the `eval` split.

For a detailed deep dive into data validation and schema generation refer to the `lab-31-tfdv-structured-data` lab.

## Preprocessing data with Transform

The `Transform` component performs data transformation and feature engineering. The `Transform` component consumes `tf.Examples` emitted from the `ExampleGen` component and emits the transformed feature data and the `SavedModel` graph that was used to process the data. The emitted `SavedModel`  can then be used by serving components to make sure that the same data pre-processing logic is applied at training and serving.

The `Transform` component requires more code than many other components because of the arbitrary complexity of the feature engineering that you may need for the data and/or model that you're working with. It requires code files to be available which define the processing needed.

<img src=../../images/Transform.png width="400">

### Define the pre-processing module

To configure `Trainsform`, you need to encapsulate your pre-processing code in the Python `preprocessing_fn` function and save it to a  python module that is then provided to the Transform component as an input. This module will be loaded by transform and the `preprocessing_fn` function will be called when the `Transform` component runs.

In most cases, your implementation of the `preprocessing_fn` makes extensive use of [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft) for performing feature engineering on your dataset.

In [26]:
TRANSFORM_MODULE = 'preprocessing.py'
!cat {TRANSFORM_MODULE}

# Copyright 2021 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype preprocessing.
This file defines a template for TFX Transform component.
"""

import tensorflow as tf
import tensorflow_transform as tft

import features

def _fill_in_missing(x):
  """Replace missing values in a SparseTensor.
  Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
  Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 

### Configure and run the `Transform` component.

In [27]:
transform = Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=TRANSFORM_MODULE)

In [28]:
context.run(transform)

INFO:absl:Running driver for Transform
INFO:absl:MetadataStore with DB connection initialized
2021-11-10 07:34:36.333581: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
2021-11-10 07:34:36.338786: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for Transform
INFO:absl:Analyze the 'train' split and transform all splits when splits_config is not set.


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`


Instructions for updating:
Schema is a deprecated, use schema_utils.schema_from_feature_spec to create a `Schema`
INFO:absl:Feature Aspect has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Cover_Type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Elevation has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_3pm has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_9am has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_Noon has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Slope has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Soil_Type has no shape. Setting to VarLenSparseTensor.
INFO:a

Instructions for updating:
Use ref() instead.


Instructions for updating:
Use ref() instead.
INFO:absl:Feature Aspect has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Cover_Type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Elevation has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_3pm has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_9am has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Hillshade_Noon has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Slope has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Soil_Type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature Vertical_Distance_To_Hydrology has no shape. Setting to 

...

value: "\n\013\n\tConst_5:0\022\tSoil_Type"



value: "\n\013\n\tConst_5:0\022\tSoil_Type"



INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 6
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 6
        type_id: 14
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Transform/transform_graph/6"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformGraph"
        )]
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 7
        type_id: 5
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Transform/transformed_examples/6"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 8
        type_id: 15
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Transform/updated_analyzer_cache/6"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 15
        name: "TransformCache"
        )]
        ))

### Examine the `Transform` component's outputs

The Transform component has 2 outputs:

- `transform_graph` - contains the graph that can perform the preprocessing operations (this graph will be included in the serving and evaluation models).
- `transformed_examples` - contains the preprocessed training and evaluation data.

Take a peek at the `transform_graph` artifact: it points to a directory containing 3 subdirectories:

In [29]:
os.listdir(transform.outputs['transform_graph'].get()[0].uri)

['metadata', 'transformed_metadata', 'transform_fn']

And the `transform.examples` artifact

In [30]:
os.listdir(transform.outputs['transformed_examples'].get()[0].uri)

['eval', 'train']

In [ ]:
transform_uri = transform.outputs['transformed_examples'].get()[0].uri
tfrecord_filenames = [os.path.join(transform_uri,  'train', name)
                      for name in os.listdir(os.path.join(transform_uri, 'train'))]
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
for tfrecord in dataset.take(2):
  example = tf.train.Example()
  example.ParseFromString(tfrecord.numpy())
  for name, feature in example.features.feature.items():
    if feature.HasField('bytes_list'):
        value = feature.bytes_list.value
    if feature.HasField('float_list'):
        value = feature.float_list.value
    if feature.HasField('int64_list'):
        value = feature.int64_list.value
    print('{}: {}'.format(name, value))
  print('******')

## Train your TensorFlow model with the `Trainer` component

The `Trainer` component trains a model using TensorFlow.

`Trainer` takes:

- tf.Examples used for training and eval.
- A user provided module file that defines the trainer logic.
- A data schema created by `SchemaGen` or imported by `ImporterNode`.
- A proto definition of train args and eval args.
- An optional transform graph produced by upstream Transform component.
- An optional base models used for scenarios such as warmstarting training.

<img src=../../images/Trainer.png width="400">


### Define the trainer module

To configure `Trainer`, you need to encapsulate your training code in a Python module that is then provided to the `Trainer` as an input. 


In [31]:
TRAINER_MODULE_FILE = 'model.py'
!cat {TRAINER_MODULE_FILE}

# Copyright 2021 Google LLC. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
"""Covertype Keras WideDeep Classifier."""

import functools
import absl
import os
from typing import List, Text

import kerastuner
import tensorflow as tf
import tensorflow_model_analysis as tfma
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import DataAc

### Create and run the Trainer component

As of the 0.25.0 release of TFX, the `Trainer` component only supports passing a single field - `num_steps` - through the `train_args` and `eval_args` arguments. 

In [32]:
trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=5000),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1000))

In [33]:
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
2021-11-10 07:36:36.564230: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
2021-11-10 07:36:36.569059: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for Trainer
INFO:absl:Training model.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shap

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                       

   1/5000 [..............................] - ETA: 0s - loss: 2.0150 - sparse_categorical_accuracy: 0.1875WARNING:tensorflow:From /opt/conda/lib/python3.7/site-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


2021-11-10 07:36:39.791176: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


...

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


2021-11-10 07:36:54.304938: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/7/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/7/serving_model_dir/assets
INFO:absl:Training complete. Model written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/7/serving_model_dir. ModelRun written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model_run/7
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 9
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 10
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model_run/7"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

### Analyzing training runs with TensorBoard

In this step you will analyze the training run with [TensorBoard.dev](https://blog.tensorflow.org/2019/12/introducing-tensorboarddev-new-way-to.html). `TensorBoard.dev` is a managed service that enables you to easily host, track and share your ML experiments.

#### Retrieve the location of TensorBoard logs

Each model run's train and eval metric logs are written to the `model_run` directory by the Tensorboard callback defined in `model.py`.

In [34]:
logs_path = trainer.outputs['model_run'].get()[0].uri
print(logs_path)

/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model_run/7


#### Upload the logs and start TensorBoard.dev

1. Open a new JupyterLab terminal window

2. From the terminal window, execute the following command
```
tensorboard dev upload --logdir [YOUR_LOGDIR]
```

Where [YOUR_LOGDIR] is an URI retrieved by the previous cell.

You will be asked to authorize `TensorBoard.dev` using your Google account. If you don't have a Google account or you don't want to authorize `TensorBoard.dev` you can skip this exercise.

After the authorization process completes, follow the link provided to view your experiment.

## Tune your model's hyperparameters with the `Tuner` component

The `Tuner` component makes use of the Python [KerasTuner](https://keras-team.github.io/keras-tuner/) API to tune your model's hyperparameters. It tighty integrates with the `Transform` and `Trainer` components for model hyperparameter tuning in continuous training pipelines as well as advanced use cases such as feature selection, feature engineering, and model architecture search.

<img src=../../images/Tuner_Overview.png width="400">

`Tuner` takes:

*  A user provided module file (or module fn) that defines the tuning logic, including model definition, hyperparameter search space, objective etc.
*  tf.Examples used for training and eval.
*  Protobuf definition of train args and eval args.
  * (Optional) Protobuf definition of tuning args.
  * (Optional) transform graph produced by an upstream Transform component.
  * (Optional) A data schema created by a SchemaGen pipeline component and optionally altered by the developer.
  
<img src=../../images/Tuner.png width="400">

With the given data, model, and objective, `Tuner` tunes the hyperparameters and emits the best results that can be directly fed into the `Trainer` component during model re-training.

In [36]:
tuner = Tuner(
        module_file=TRAINER_MODULE_FILE,
        examples=transform.outputs['transformed_examples'],
        transform_graph=transform.outputs['transform_graph'],
        train_args=trainer_pb2.TrainArgs(num_steps=1000),
        eval_args=trainer_pb2.EvalArgs(num_steps=500))

In [37]:
context.run(tuner)

INFO:absl:Running driver for Tuner
INFO:absl:MetadataStore with DB connection initialized
2021-11-10 07:45:47.428200: W ml_metadata/metadata_store/rdbms_metadata_access_object.cc:581] No property is defined for the Type
INFO:absl:Running executor for Tuner
INFO:absl:Creating temp directory at /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/.temp/8/
INFO:absl:Train on the 'train' split when train_args.splits is not set.
INFO:absl:Evaluate on the 'eval' split when eval_args.splits is not set.
INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:____

INFO:absl:Feature Slope_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Soil_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Vertical_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Wilderness_Area_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways_xf has a shape . Setting to DenseTensor.

INFO:absl:Start tuning... Tuner ID: tuner0
INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                            
INFO:absl:___________________________________________

1000/1000 [==============================] - ETA: 0s - loss: 2.0868 - sparse_categorical_accuracy: 0.109 - ETA: 1s - loss: 1.0440 - sparse_categorical_accuracy: 0.615 - ETA: 1s - loss: 0.9154 - sparse_categorical_accuracy: 0.645 - ETA: 1s - loss: 0.8679 - sparse_categorical_accuracy: 0.655 - ETA: 1s - loss: 0.8340 - sparse_categorical_accuracy: 0.660 - ETA: 1s - loss: 0.8104 - sparse_categorical_accuracy: 0.666 - ETA: 1s - loss: 0.7911 - sparse_categorical_accuracy: 0.674 - ETA: 1s - loss: 0.7719 - sparse_categorical_accuracy: 0.681 - ETA: 1s - loss: 0.7617 - sparse_categorical_accuracy: 0.683 - ETA: 1s - loss: 0.7536 - sparse_categorical_accuracy: 0.684 - ETA: 1s - loss: 0.7470 - sparse_categorical_accuracy: 0.686 - ETA: 1s - loss: 0.7413 - sparse_categorical_accuracy: 0.688 - ETA: 0s - loss: 0.7366 - sparse_categorical_accuracy: 0.689 - ETA: 0s - loss: 0.7312 - sparse_categorical_accuracy: 0.691 - ETA: 0s - loss: 0.7300 - sparse_categorical_accuracy: 0.691 - ETA: 0s - loss: 0.7250 - 

INFO:absl:Model: "functional_1"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                            
INFO:absl:______________________________________________________________________________________

...

1000/1000 [==============================] - ETA: 0s - loss: 1.7352 - sparse_categorical_accuracy: 0.359 - ETA: 1s - loss: 1.8777 - sparse_categorical_accuracy: 0.253 - ETA: 1s - loss: 1.7364 - sparse_categorical_accuracy: 0.358 - ETA: 1s - loss: 1.6303 - sparse_categorical_accuracy: 0.429 - ETA: 1s - loss: 1.5318 - sparse_categorical_accuracy: 0.483 - ETA: 1s - loss: 1.4532 - sparse_categorical_accuracy: 0.517 - ETA: 1s - loss: 1.3852 - sparse_categorical_accuracy: 0.539 - ETA: 1s - loss: 1.3245 - sparse_categorical_accuracy: 0.559 - ETA: 1s - loss: 1.2752 - sparse_categorical_accuracy: 0.573 - ETA: 1s - loss: 1.2328 - sparse_categorical_accuracy: 0.585 - ETA: 0s - loss: 1.1983 - sparse_categorical_accuracy: 0.594 - ETA: 0s - loss: 1.1724 - sparse_categorical_accuracy: 0.599 - ETA: 0s - loss: 1.1431 - sparse_categorical_accuracy: 0.608 - ETA: 0s - loss: 1.1153 - sparse_categorical_accuracy: 0.614 - ETA: 0s - loss: 1.0915 - sparse_categorical_accuracy: 0.619 - ETA: 0s - loss: 1.0710 - 

INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit
INFO:absl:Finished tuning... Tuner ID: tuner0


INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.001, 'values': [0.01, 0.001, 0.0001], 'ordered': True}}, {'class_name': 'Int', 'config': {'name': 'n_layers', 'default': 1, 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'n_layers=1/n_units_1', 'default': 8, 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'n_layers=2/n_units_1', 'default': 8, 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}}, {'class_name': 'Int', 'config': {'name': 'n_layers=2/n_units_2', 'default': 8, 'min_value': 8, 'max_value': 128, 'step': 8, 'sampling': None}}], 'values': {'learning_rate': 0.01, 'n_layers': 1, 'n_layers=1/n_units_1': 96, 'n_layers=2/n_units_1': 16, 'n_layers=2/n_units_2': 16}}
INFO:absl:Best Hyperparameters are written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Tuner/best_hy

ExecutionResult(
    component_id: Tuner
    execution_id: 8
    outputs:
        best_hyperparameters: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 11
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Tuner/best_hyperparameters/8"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

### Retrain your model by running Tuner with the best hyperparameters

In [38]:
hparams_importer = ImporterNode(
    instance_name='import_hparams',
    # This can be Tuner's output file or manually edited file. The file contains
    # text format of hyperparameters (kerastuner.HyperParameters.get_config())
    source_uri=tuner.outputs.best_hyperparameters.get()[0].uri,
    artifact_type=HyperParameters)

In [39]:
context.run(hparams_importer)

INFO:absl:Running driver for ImporterNode.import_hparams
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Processing source uri: /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Tuner/best_hyperparameters/8, properties: {}, custom_properties: {}
INFO:absl:Reusing existing artifact
INFO:absl:Running executor for ImporterNode.import_hparams
INFO:absl:Running publisher for ImporterNode.import_hparams
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ImporterNode.import_hparams
    execution_id: 9
    outputs:
        result: Channel(
            type_name: HyperParameters
            artifacts: [Artifact(artifact: id: 11
        type_id: 20
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Tuner/best_hyperparameters/8"
        custom_properties {
          key: "name"
          value {
            string_value: "best_hyperparameters"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Tuner"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        create_time_since_epoch: 1636530347434
        last_update_time_since_epoch: 1636530388158
        , artifact_type: id: 20
        name: "HyperParameters"
        )]
        ))

In [43]:
# TODO: your code to retrain your model with the best hyperparameters found by the Tuner component above.
# Hint: review the Trainer code above in this notebook and the documentation for how to configure the trainer 
# to use the output artifact from the hparams_importer.

trainer = Trainer(
    custom_executor_spec=executor_spec.ExecutorClassSpec(trainer_executor.GenericExecutor),
    module_file=TRAINER_MODULE_FILE,
    transformed_examples=transform.outputs.transformed_examples,
    schema=schema_importer.outputs.result,
    transform_graph=transform.outputs.transform_graph,
    hyperparameters=hparams_importer.outputs.result,    
    train_args=trainer_pb2.TrainArgs(splits=['train'], num_steps=2),
    eval_args=trainer_pb2.EvalArgs(splits=['eval'], num_steps=1)
)

In [53]:
context.run(trainer)

INFO:absl:Running driver for Trainer
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Trainer
INFO:absl:Training model.
INFO:absl:Feature Aspect_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Cover_Type_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Elevation_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_3pm_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_9am_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Hillshade_Noon_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Fire_Points_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Hydrology_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Horizontal_Distance_To_Roadways_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Slope_xf has a shape . Setting to DenseTensor.
INFO:absl:Feature Soil_Type_xf has a shape . Setting to DenseTensor.
INFO:ab

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:absl:Model: "functional_3"
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Layer (type)                    Output Shape         Param #     Connected to                     
INFO:absl:==================================================================================================
INFO:absl:Aspect_xf (InputLayer)          [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Elevation_xf (InputLayer)       [(None,)]            0                                            
INFO:absl:__________________________________________________________________________________________________
INFO:absl:Hillshade_3pm_xf (InputLayer)   [(None,)]            0                                       

1/2 [==============>...............] - ETA: 0s - loss: 2.0727 - sparse_categorical_accuracy: 0.1406

2021-11-10 07:59:33.946209: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session started.


...

2/2 [==============================] - 2s 1s/step - loss: 1.9841 - sparse_categorical_accuracy: 0.2188 - val_loss: 1.6740 - val_sparse_categorical_accuracy: 0.3594
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14/serving_model_dir/assets


INFO:tensorflow:Assets written to: /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14/serving_model_dir/assets
INFO:absl:Training complete. Model written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14/serving_model_dir. ModelRun written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model_run/14
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 14
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 16
        type_id: 17
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "Model"
        )]
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 17
        type_id: 18
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model_run/14"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 18
        name: "ModelRun"
        )]
        ))

## Evaluating trained models with Evaluator
The `Evaluator` component analyzes model performance using the [TensorFlow Model Analysis library](https://www.tensorflow.org/tfx/model_analysis/get_started). It runs inference requests on particular subsets of the test dataset, based on which slices are defined by the developer. Knowing which slices should be analyzed requires domain knowledge of what is important in this particular use case or domain. 

The `Evaluator` can also optionally validate a newly trained model against a previous model. In this lab, you only train one model, so the Evaluator automatically will label the model as "blessed".


<img src=../../images/Evaluator.png width="400">

### Configure and run the Evaluator component


Use the `ResolverNode` to pick the previous model to compare against.  The model resolver is only required if performing model validation in addition to evaluation. In this case we validate against the latest blessed model. If no model has been blessed before (as in this case) the evaluator will make our candidate the first blessed model.

In [54]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))

In [55]:
context.run(model_resolver)

INFO:absl:Running driver for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for ResolverNode.latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ResolverNode.latest_blessed_model_resolver
    execution_id: 15
    outputs:
        model: Channel(
            type_name: Model
            artifacts: []
        )
        model_blessing: Channel(
            type_name: ModelBlessing
            artifacts: []
        ))

Configure evaluation metrics and slices.

In [75]:
# TODO: Your code here to create a tfma.MetricThreshold. 
# Review the API documentation here: https://www.tensorflow.org/tfx/model_analysis/api_docs/python/tfma/MetricThreshold
# Hint: Review the API documentation for tfma.GenericValueThreshold to constrain accuracy between 50% and 99%.

accuracy_threshold = tfma.MetricThreshold(
                value_threshold=tfma.GenericValueThreshold(
                    lower_bound={'value': 0.20},
                    upper_bound={'value': 0.99})
                )

metrics_specs = tfma.MetricsSpec(
                   metrics = [
                       tfma.MetricConfig(class_name='SparseCategoricalAccuracy',
                           threshold=accuracy_threshold),
                       tfma.MetricConfig(class_name='ExampleCount')])

eval_config = tfma.EvalConfig(
    model_specs=[
        tfma.ModelSpec(label_key='Cover_Type')
    ],
    metrics_specs=[metrics_specs],
    slicing_specs=[
        tfma.SlicingSpec(),
        tfma.SlicingSpec(feature_keys=['Wilderness_Area'])
    ]
)
eval_config

model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.2
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

In [76]:
trainer.outputs.model

Channel(
    type_name: Model
    artifacts: [Artifact(artifact: id: 16
type_id: 17
uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14"
custom_properties {
  key: "name"
  value {
    string_value: "model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Trainer"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 17
name: "Model"
)]
)

In [77]:
model_analyzer = Evaluator(
    examples=example_gen.outputs.examples,
    model=trainer.outputs.model,
    baseline_model=model_resolver.outputs.model,
    eval_config=eval_config
)

In [78]:
model_analyzer

Evaluator(spec: <tfx.types.standard_component_specs.EvaluatorSpec object at 0x7f0ecf843b50>, executor_spec: <tfx.dsl.components.base.executor_spec.ExecutorClassSpec object at 0x7f0ecf80c690>, driver_class: <class 'tfx.dsl.components.base.base_driver.BaseDriver'>, component_id: Evaluator, inputs: {'examples': Channel(
    type_name: Examples
    artifacts: [Artifact(artifact: id: 1
type_id: 5
uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/CsvExampleGen/examples/1"
properties {
  key: "split_names"
  value {
    string_value: "[\"train\", \"eval\"]"
  }
}
custom_properties {
  key: "input_fingerprint"
  value {
    string_value: "split:single_split,num_files:1,total_bytes:5511886,xor_checksum:1585681978,sum_checksum:1585681978"
  }
}
custom_properties {
  key: "payload_format"
  value {
    string_value: "FORMAT_TF_EXAMPLE"
  }
}
custom_properties {
  key: "span"
  value {
    string_value: "0"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 5
name: "Examples"
properties {
  key: "span"
  value: INT
}
properties {
  key: "split_names"
  value: STRING
}
properties {
  key: "version"
  value: INT
}
)]
), 'model': Channel(
    type_name: Model
    artifacts: [Artifact(artifact: id: 16
type_id: 17
uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14"
custom_properties {
  key: "name"
  value {
    string_value: "model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Trainer"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 17
name: "Model"
)]
), 'baseline_model': Channel(
    type_name: Model
    artifacts: []
)}, outputs: {'evaluation': Channel(
    type_name: ModelEvaluation
    artifacts: []
), 'blessing': Channel(
    type_name: ModelBlessing
    artifacts: []
)})

In [79]:
context.run(
    model_analyzer, 
    enable_cache=False
)

INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
        please use "has_baseline" instead.
INFO:absl:Request was made to ignore the baseline ModelSpec and any change thresholds. This is likely because a baseline model was not provided: updated_config=
model_specs {
  label_key: "Cover_Type"
}
slicing_specs {
}
slicing_specs {
  feature_keys: "Wilderness_Area"
}
metrics_specs {
  metrics {
    class_name: "SparseCategoricalAccuracy"
    threshold {
      value_threshold {
        lower_bound {
          value: 0.2
        }
        upper_bound {
          value: 0.99
        }
      }
    }
  }
  metrics {
    class_name: "ExampleCount"
  }
}

INFO:absl:Using /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14/serving_model_dir as  model.


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.


INFO:absl:Evaluation complete. Results written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Evaluator/evaluation/19.
INFO:absl:Checking validation results.
INFO:absl:Blessing result True written to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Evaluator/blessing/19.
INFO:absl:Running publisher for Evaluator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Evaluator
    execution_id: 19
    outputs:
        evaluation: Channel(
            type_name: ModelEvaluation
            artifacts: [Artifact(artifact: id: 22
        type_id: 23
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Evaluator/evaluation/19"
        custom_properties {
          key: "name"
          value {
            string_value: "evaluation"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 23
        name: "ModelEvaluation"
        )]
        )
        blessing: Channel(
            type_name: ModelBlessing
            artifacts: [Artifact(artifact: id: 23
        type_id: 24
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Evaluator/blessing/19"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "current_model"
          value {
            string_value: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14"
          }
        }
        custom_properties {
          key: "current_model_id"
          value {
            int_value: 16
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Evaluator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 24
        name: "ModelBlessing"
        )]
        ))

### Check the model performance validation status

In [80]:
model_blessing_uri = model_analyzer.outputs.blessing.get()[0].uri
!ls -l {model_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Nov 10 08:07 BLESSED


### Visualize evaluation results
You can visualize the evaluation results using the `tfma.view.render_slicing_metrics()` function from TensorFlow Model Analysis library.

**Setup Note:** *Currently, TFMA visualizations don't render in  JupyterLab. Make sure that you run this notebook in Classic Notebook.*

In [81]:
evaluation_uri = model_analyzer.outputs['evaluation'].get()[0].uri
evaluation_uri
!ls {evaluation_uri}

eval_config.json  metrics  plots  validations


In [82]:
eval_result = tfma.load_eval_result(evaluation_uri)
eval_result

EvalResult(slicing_metrics=[((), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.3885249197483063}, 'example_count': {'doubleValue': 20148.0}}}}), ((('Wilderness_Area', 'Cache'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.42920681834220886}, 'example_count': {'doubleValue': 1349.0}}}}), ((('Wilderness_Area', 'Commanche'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.26618215441703796}, 'example_count': {'doubleValue': 8806.0}}}}), ((('Wilderness_Area', 'Rawah'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.505638062953949}, 'example_count': {'doubleValue': 8957.0}}}}), ((('Wilderness_Area', 'Neota'),), {'': {'': {'sparse_categorical_accuracy': {'doubleValue': 0.3629343509674072}, 'example_count': {'doubleValue': 1036.0}}}})], plots=[((), None), ((('Wilderness_Area', 'Cache'),), None), ((('Wilderness_Area', 'Commanche'),), None), ((('Wilderness_Area', 'Rawah'),), None), ((('Wilderness_Area', 'Neota'),), None)], config=mode

In [83]:
tfma.view.render_slicing_metrics(eval_result)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Overall', 'metrics':…

In [84]:
tfma.view.render_slicing_metrics(
    eval_result, slicing_column='Wilderness_Area')

SlicingMetricsViewer(config={'weightedExamplesColumn': 'example_count'}, data=[{'slice': 'Wilderness_Area:Cach…

## InfraValidator

The `InfraValidator` component acts as an additional early warning layer by validating a candidate model in a sandbox version of its serving infrastructure to prevent an unservable model from being pushed to production. Compared to the `Evaluator` component above which validates a model's performance, the `InfraValidator` component is validating that a model is able to generate predictions from served examples in an environment configured to match production. The config below takes a model and examples, launches the model in a sand-boxed [TensorflowServing](https://www.tensorflow.org/tfx/guide/serving) model server from the latest image in a local docker engine, and optionally checks that the model binary can be loaded and queried before "blessing" it for production.

<img src=../../images/InfraValidator.png width="400">

In [85]:
infra_validator = InfraValidator(
    model=trainer.outputs['model'],
    examples=example_gen.outputs['examples'],
    serving_spec=infra_validator_pb2.ServingSpec(
        tensorflow_serving=infra_validator_pb2.TensorFlowServing(
            tags=['latest']),
      local_docker=infra_validator_pb2.LocalDockerConfig(),
  ),
    validation_spec=infra_validator_pb2.ValidationSpec(
        max_loading_time_seconds=60,
        num_tries=5,
    ),    
  request_spec=infra_validator_pb2.RequestSpec(
      tensorflow_serving=infra_validator_pb2.TensorFlowServingRequestSpec(),
          num_examples=5,
      )
)

In [86]:
context.run(infra_validator, enable_cache=False)

INFO:absl:Running driver for InfraValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for InfraValidator
INFO:absl:InfraValidator will be run in LOAD_AND_QUERY mode.
INFO:absl:tag_set is not given. Using {'serve'} instead.
INFO:absl:signature_names are not given. Using ['serving_default'] instead.
INFO:absl:Creating temp directory at /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/.temp/20/
INFO:absl:Starting infra validation (attempt 1/5).
INFO:absl:Starting LocalDockerRunner(image: tensorflow/serving:latest).
INFO:absl:Running container with parameter {'auto_remove': True, 'detach': True, 'publish_all_ports': True, 'image': 'tensorflow/serving:latest', 'environment': {'MODEL_NAME': 'infra-validation-model', 'MODEL_BASE_PATH': '/model'}, 'mounts': [{'Target': '/model/infra-validation-model/1', 'Source': '/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/.temp/20/infra-validation-model/1636531621', 'Type'

ExecutionResult(
    component_id: InfraValidator
    execution_id: 20
    outputs:
        blessing: Channel(
            type_name: InfraBlessing
            artifacts: [Artifact(artifact: id: 24
        type_id: 26
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/InfraValidator/blessing/20"
        custom_properties {
          key: "blessed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "blessing"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "InfraValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 26
        name: "InfraBlessing"
        )]
        ))

### Check the model infrastructure validation status

In [87]:
infra_blessing_uri = infra_validator.outputs.blessing.get()[0].uri
!ls -l {infra_blessing_uri}

total 0
-rw-r--r-- 1 jupyter jupyter 0 Nov 10 08:07 INFRA_BLESSED


## Deploying models with Pusher

The `Pusher` component checks whether a model has been "blessed", and if so, deploys it by pushing the model to a well known file destination.

<img src=../../images/Pusher.png width="400">



### Configure and run the `Pusher` component

In [88]:
trainer.outputs['model']

Channel(
    type_name: Model
    artifacts: [Artifact(artifact: id: 16
type_id: 17
uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Trainer/model/14"
custom_properties {
  key: "name"
  value {
    string_value: "model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Trainer"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 17
name: "Model"
)]
)

In [89]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=model_analyzer.outputs['blessing'],
    infra_blessing=infra_validator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=SERVING_MODEL_DIR)))
context.run(pusher)

INFO:absl:Running driver for Pusher
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Pusher
INFO:absl:Model version: 1636531634
INFO:absl:Model written to serving path /home/jupyter/serving_model/1636531634.
INFO:absl:Model pushed to /home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Pusher/pushed_model/21.
INFO:absl:Running publisher for Pusher
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Pusher
    execution_id: 21
    outputs:
        pushed_model: Channel(
            type_name: PushedModel
            artifacts: [Artifact(artifact: id: 25
        type_id: 28
        uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Pusher/pushed_model/21"
        custom_properties {
          key: "name"
          value {
            string_value: "pushed_model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Pusher"
          }
        }
        custom_properties {
          key: "pushed"
          value {
            int_value: 1
          }
        }
        custom_properties {
          key: "pushed_destination"
          value {
            string_value: "/home/jupyter/serving_model/1636531634"
          }
        }
        custom_properties {
          key: "pushed_version"
          value {
            string_value: "1636531634"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        state: LIVE
        , artifact_type: id: 28
        name: "PushedModel"
        )]
        ))

### Examine the output of `Pusher`

In [90]:
pusher.outputs

{'pushed_model': Channel(
    type_name: PushedModel
    artifacts: [Artifact(artifact: id: 25
type_id: 28
uri: "/home/jupyter/artifact-store/tfx-covertype-classifier/20211110_072117/Pusher/pushed_model/21"
custom_properties {
  key: "name"
  value {
    string_value: "pushed_model"
  }
}
custom_properties {
  key: "producer_component"
  value {
    string_value: "Pusher"
  }
}
custom_properties {
  key: "pushed"
  value {
    int_value: 1
  }
}
custom_properties {
  key: "pushed_destination"
  value {
    string_value: "/home/jupyter/serving_model/1636531634"
  }
}
custom_properties {
  key: "pushed_version"
  value {
    string_value: "1636531634"
  }
}
custom_properties {
  key: "state"
  value {
    string_value: "published"
  }
}
state: LIVE
, artifact_type: id: 28
name: "PushedModel"
)]
)}

In [91]:
# Set `PATH` to include a directory containing `saved_model_cli.
PATH=%env PATH
%env PATH=/opt/conda/envs/tfx/bin:{PATH}

env: PATH=/opt/conda/envs/tfx/bin:/opt/conda/envs/tfx/bin:/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [92]:
SERVING_MODEL_DIR

'/home/jupyter/serving_model'

In [93]:
latest_pushed_model = os.path.join(SERVING_MODEL_DIR, max(os.listdir(SERVING_MODEL_DIR)))
!saved_model_cli show --dir {latest_pushed_model} --all

2021-11-10 08:07:19.429624: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2021-11-10 08:07:19.429665: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.

MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['exam

## Next steps

This concludes your introductory walthrough through TFX pipeline components. In the lab, you used TFX to analyze, understand, and pre-process the dataset and train, analyze, validate, and deploy a multi-class classification model to predict the type of forest cover from cartographic features. You utilized a TFX Interactive Context for prototype development of a TFX pipeline directly in a Jupyter notebook. Next, you worked with the TFDV library to modify your dataset schema to add feature constraints to catch data anamolies that can negatively impact your model's performance. You utilized TFT library for feature proprocessing for consistent feature transformations for your model at training and serving time. Lastly, using the TFMA library, you added model performance constraints to ensure you only push more accurate models than previous runs to production.


The next labs in the series will guide through developing a TFX pipeline, deploying and running the pipeline on **AI Platform Pipelines** and automating the pipeline build and deployment processes with **Cloud Build**.

## License

Copyright 2021 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.